# SuAVE Simple Variable Calculations
### This sample notebook will read numeric variables from a survey dataset and let users compute a derivative numeric variable and add it to new survey version

### 1. Retrieve survey parameters from the URL

In [616]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 


<IPython.core.display.Javascript object>

### 2. Read the survey file and extract numeric variables

In [617]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd


absolutePath = "../../temp_csvs/"

# read the csv file
file = open(absolutePath + csv_file, encoding="latin-1")
print(absolutePath + csv_file)
df = pd.read_csv(file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
print(variables_df.varname.values)

# create a dictionary of #number variables with abbreviated and full variable names 
var_list = {n[:n.index('#')]:n for n in variables_df.varname.values if '#number' in n}
print(var_list)

IsADirectoryError: [Errno 21] Is a directory: '../../temp_csvs/'

### 3. Define a new variable using the form ##

In [618]:
# Define a new Variable
a = widgets.Dropdown(options=var_list.keys())
b = widgets.Dropdown(options=['/','*'])
c = widgets.Dropdown(options=var_list.keys())
ui = widgets.VBox([a, b, c])
def f(a, b, c):
    return ((a, b, c))

formula = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, formula)

print("Select variables and the operation, then run the next cell")

Output()

Select variables and the operation, then run the next cell


### 4. Edit variable name as needed

In [609]:
# Give a Name to the New Variable
def f(Var_Name):
    return Var_Name
newvar = interact(f, Var_Name=a.value + b.value + c.value +'#number');
print("After defining variable name hit Enter, then run the next cell")

interactive(children=(Text(value='Ad Clicks/Ad Impressions#number', description='Var_Name'), Output()), _dom_c…

After defining variable name hit Enter, then run the next cell


### 5. Compute the new variable and format it for SuAVE

In [610]:
#Open the selected notebook
# In [73]:
# Compute the new variable

if b.value == '/':
    df[newvar.widget.result] = df[var_list[a.value]] / df[var_list[c.value]]
elif b.value == '*':
    df[newvar.widget.result] = df[var_list[a.value]] * df[var_list[c.value]]
elif b.value == '+':
    df[newvar.widget.result] = df[var_list[a.value]] + df[var_list[c.value]]
elif b.value == '-':
    df[newvar.widget.result] = df[var_list[a.value]] - df[var_list[c.value]]
    
# make sure there are no illegal NaN type values in this #number variable
df[newvar.widget.result].fillna('',inplace=True)
df[newvar.widget.result] = pd.to_numeric(df[newvar.widget.result], errors='coerce', downcast='float')
df[newvar.widget.result] = df[newvar.widget.result].apply(lambda x: '{:.6f}'.format(x))
df.replace(['None', 'nan'], pd.np.nan, inplace=True)

### 6. Save the new version of CSV file, and give a name to new survey

In [611]:
# new filename
new_file = absolutePath + csv_file[:-4]+'_v1.csv'
print(new_file)
df.to_csv(new_file, index=None)

../../temp_csvs/zaslavsk_Facebook_Ads_from_Russia_v1.csv


In [612]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

print("Input survey name here, then press Enter:")
# Display input text box widget for input
display(input_text)

display(output_text)
print('After setting new survey name, run the next cell')

Input survey name here, then press Enter:


Text(value='')

Text(value='')

After setting new survey name, run the next cell


In [613]:
#Print survey name
survey_name = output_text.value
print("Survey name is:", survey_name)

Survey name is: FBook_v25


### 7. Generate the survey and create survey URL

In [614]:
import requests
upload_url = "http://suave.sdsc.edu:3001/uploadCSV"
#upload_url = "http://localhost:3001/uploadCSV"
upload_data = {'name': survey_name, 'dzc': dzc_file, 'user':user}
files = {"file": open(new_file, "rb")}
r = requests.post(upload_url, files=files, data=upload_data)
print(r.status_code, r.reason)

regex = re.compile('[^0-9a-zA-Z_]')
survey_url = survey_name
survey_url =  regex.sub('_', survey_url)

url = "https://suave-dev.sdsc.edu/main/file=" + user + "_" + survey_url + ".csv" + "&views=1110001&view=grid"
print(url)
print ("Click the URL to open the new survey")


200 OK
https://suave-dev.sdsc.edu/main/file=zaslavsk_FBook_v25.csv&views=1110001&view=grid
Click the URL to open the new survey
